In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from statsmodels.tsa.ar_model import AutoReg as AR
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
import seaborn as sns

def load_data(file_path):
    """Load and prepare the pollution dataset."""
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

# Load the data
df = load_data('Pollution.csv')

In [ ]:
def load_and_prepare_data(file_path):
    """Load and prepare the pollution dataset."""
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def plot_time_series(df):
    """Create a line plot of PM2.5 values over time."""
    plt.figure(figsize=(12, 6))
    # Convert to numpy array before plotting
    plt.plot(df.index.to_numpy(), df['pm2.5'].to_numpy())
    plt.title('Pollution over Time')
    plt.xlabel('Date')
    plt.ylabel('PM2.5')
    plt.grid(True)
    plt.show()
    
    print("Observations from the time series plot:")
    print("1. The data shows high variability with significant peaks and troughs.")
    print("2. There appears to be some seasonality in the pollution levels.")
    print("3. Some extreme values (spikes) can be observed throughout the time period.")
    print("4. There are periods with missing data (shown as gaps or zeros in the plot).")

def compute_lag_correlation(df):
    """Generate lagged time sequence and compute Pearson correlation."""
    # Create lagged series
    df['lag_1'] = df['pm2.5'].shift(1)
    
    # Drop NaN values
    df_clean = df.dropna()
    
    # Compute Pearson correlation
    correlation, p_value = pearsonr(df_clean['pm2.5'], df_clean['lag_1'])
    
    print(f"Pearson correlation coefficient for one-day lag: {correlation:.4f}")
    print(f"P-value: {p_value:.10f}")
    
    return df_clean, correlation

def plot_lag_scatter(df_clean):
    """Create scatter plot between original and lagged values."""
    plt.figure(figsize=(10, 6))
    # Convert to numpy arrays before plotting
    plt.scatter(df_clean['lag_1'].to_numpy(), df_clean['pm2.5'].to_numpy(), alpha=0.5, color='blue')
    plt.title('Scatter Plot: Current vs. Lagged PM2.5 Values')
    plt.xlabel('PM2.5 at t-1')
    plt.ylabel('PM2.5 at t')
    plt.grid(True)
    
    # Add a line of best fit
    z = np.polyfit(df_clean['lag_1'].to_numpy(), df_clean['pm2.5'].to_numpy(), 1)
    p = np.poly1d(z)
    plt.plot(df_clean['lag_1'].to_numpy(), p(df_clean['lag_1'].to_numpy()), "r--", alpha=0.8)
    
    plt.show()
    
    print("Inference from scatter plot:")
    print("1. There is a strong positive correlation between current and lagged PM2.5 values.")
    print("2. Points cluster along the diagonal, indicating high autocorrelation.")
    print("3. The scatter pattern confirms the high Pearson correlation coefficient.")

def plot_autocorrelation(df):
    """Plot the autocorrelation function up to 100 lags."""
    plt.figure(figsize=(12, 6))
    plot_acf(df['pm2.5'].dropna(), lags=100)
    plt.title('Autocorrelation Function for PM2.5')
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.grid(True)
    plt.show()
    
    print("Observations from ACF plot:")
    print("1. The autocorrelation decreases as the lag increases.")
    print("2. Significant autocorrelation exists even at higher lag values.")
    print("3. The slow decay suggests the time series has strong persistence.")
    print("4. There may be seasonal patterns as indicated by periodic increases in correlation.")


In [ ]:
df = load_and_prepare_data('Pollution.csv')
print("Data loaded successfully. First few rows:")
print(df.head())

# Part 1: Autocorrelation analysis
print("\n--- Part 1: Autocorrelation Analysis ---")
plot_time_series(df)
df_clean, correlation = compute_lag_correlation(df)
plot_lag_scatter(df_clean)
plot_autocorrelation(df)


In [ ]:
def split_data(df, train_size=0.65):
    """Split the data into training and testing sets."""
    train_size = int(len(df) * train_size)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]
    return train, test

def train_and_predict_ar(train, test, lag):
    """Train AR model and make predictions for the test set."""
    # Train the model
    model = AR(train['pm2.5'], lags=lag).fit()
    
    # Get the coefficients
    coef = model.params
    print(f"AR({lag}) Model Coefficients: {coef}")
    
    # Prepare for prediction
    history = list(train['pm2.5'].values)
    
    # Make predictions
    predictions = []
    for t in range(len(test)):
        lag_values = history[-lag:]
        # Ensure we have enough lag values
        if len(lag_values) < lag:
            # Pad with zeros if needed
            lag_values = [0] * (lag - len(lag_values)) + lag_values
            
        # Compute prediction using the AR formula
        yhat = coef[0]
        for i in range(lag):
            if i < len(coef) - 1:  # Check if we have coefficient for this lag
                yhat += coef[i+1] * lag_values[lag-i-1]
        
        predictions.append(yhat)
        history.append(test['pm2.5'].iloc[t])
    
    return predictions, coef

def plot_predictions(test, predictions, lag):
    """Create scatter and line plots for actual vs. predicted values."""
    # Scatter plot - convert to numpy arrays before plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(test['pm2.5'].to_numpy(), predictions, alpha=0.5)
    plt.title(f'Actual vs. Predicted PM2.5 (Lag={lag})')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.grid(True)
    
    # Add a diagonal line for reference
    min_val = min(min(test['pm2.5']), min(predictions))
    max_val = max(max(test['pm2.5']), max(predictions))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    plt.show()
    
    # Line plot - convert to numpy arrays before plotting
    plt.figure(figsize=(12, 6))
    plt.plot(test.index.to_numpy(), test['pm2.5'].to_numpy(), label='Actual')
    plt.plot(test.index.to_numpy(), predictions, label='Predicted', alpha=0.7)
    plt.title(f'Actual vs. Predicted PM2.5 Over Time (Lag={lag})')
    plt.xlabel('Date')
    plt.ylabel('PM2.5')
    plt.legend()
    plt.grid(True)
    plt.show()

def evaluate_model(test, predictions):
    """Calculate RMSE and MAPE for model evaluation."""
    rmse = np.sqrt(mean_squared_error(test['pm2.5'], predictions))
    mape = mean_absolute_percentage_error(test['pm2.5'], predictions) * 100
    
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}%")
    
    return rmse, mape

def run_multiple_ar_models(train, test, lag_values):
    """Run AR models with different lag values and compare results."""
    results = []
    
    for lag in lag_values:
        print(f"\nRunning AR model with lag = {lag}")
        predictions, _ = train_and_predict_ar(train, test, lag)
        
        # Evaluate the model
        rmse, mape = evaluate_model(test, predictions)
        
        # Store results
        results.append({
            'lag': lag,
            'rmse': rmse,
            'mape': mape,
            'predictions': predictions
        })
        
        plot_predictions(test, predictions, lag)
    
    return results

def plot_performance_metrics(results):
    """Plot RMSE and MAPE for different lag values."""
    lags = [r['lag'] for r in results]
    rmse_values = [r['rmse'] for r in results]
    mape_values = [r['mape'] for r in results]
    
    # RMSE plot
    plt.figure(figsize=(10, 6))
    plt.bar(lags, rmse_values)
    plt.title('RMSE vs. Lag Values')
    plt.xlabel('Lag')
    plt.ylabel('RMSE')
    plt.xticks(lags)
    plt.grid(True, axis='y')
    plt.show()
    
    # MAPE plot
    plt.figure(figsize=(10, 6))
    plt.bar(lags, mape_values)
    plt.title('MAPE vs. Lag Values')
    plt.xlabel('Lag')
    plt.ylabel('MAPE (%)')
    plt.xticks(lags)
    plt.grid(True, axis='y')
    plt.show()
    
    print("Inference from RMSE and MAPE charts:")
    print("1. The error metrics tend to decrease as lag values increase initially.")
    print("2. After a certain point, adding more lags doesn't significantly improve performance.")
    print("3. There may be an optimal lag value that balances model complexity and accuracy.")

In [ ]:
# Part 2: Autoregression models
print("\n--- Part 2: Autoregression Models ---")
train, test = split_data(df)
print(f"Training set size: {len(train)}")
print(f"Testing set size: {len(test)}")

# Run models with different lag values
lag_values = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
results = run_multiple_ar_models(train, test, lag_values)
plot_performance_metrics(results)

In [ ]:
def find_optimal_lag(train):
    """Find optimal lag using the heuristic abs(AutoCorrelation) > 2/sqrt(T)."""
    T = len(train)
    threshold = 2 / np.sqrt(T)
    
    # Calculate autocorrelation for different lags
    acf_values = []
    max_lag = 100  # Maximum lag to consider
    
    for lag in range(1, max_lag + 1):
        # Create lagged series
        original = train['pm2.5'].iloc[lag:].values  # Skip first 'lag' values
        lagged = train['pm2.5'].iloc[:-lag].values   # Skip last 'lag' values
        
        # Ensure both arrays have the same length
        if len(original) > 0 and len(lagged) > 0:
            corr, _ = pearsonr(original, lagged)
            acf_values.append((lag, abs(corr)))
    
    # Find the maximum lag that satisfies the condition
    optimal_lag = 1
    for lag, acf in acf_values:
        if acf > threshold:
            optimal_lag = lag
    
    print(f"Threshold for significance: {threshold:.4f}")
    print(f"Optimal lag based on heuristic: {optimal_lag}")
    
    return optimal_lag

In [ ]:
# Part 3: Optimal lag selection
print("\n--- Part 3: Optimal Lag Selection ---")
optimal_lag = find_optimal_lag(train)

# Run model with optimal lag
print(f"\nRunning AR model with optimal lag = {optimal_lag}")
predictions, _ = train_and_predict_ar(train, test, optimal_lag)
plot_predictions(test, predictions, optimal_lag)
rmse, mape = evaluate_model(test, predictions)

# Compare with results from Part 2
print("\nComparison with models from Part 2:")
print(f"Optimal lag model - RMSE: {rmse:.4f}, MAPE: {mape:.4f}%")
for result in results:
    print(f"Lag {result['lag']} model - RMSE: {result['rmse']:.4f}, MAPE: {result['mape']:.4f}%")